In [1]:
import json
import numpy
import re
import os
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec



def load_data(input_file):
    '''
    input: result_13k.json
    output: rep_list,dep_list
    '''
    with open(input_file) as f:
        data = json.load(f)
    print(f'repos nums: {len(data)}')

    need_to_remove = []
    for k,v in data.items():
        if 'No dependency' in v:
            need_to_remove.append(k)
    print(f'repos which have no dependency files: {len(need_to_remove)}')

    for k in need_to_remove:
        del data[k]
    print(f'repos with dependency files: {len(data)}')

    rep_list,dep_list = [],[]
    for k,v in data.items():
        rep_list.append(k)
        dep_list.append(v)
        
    dep_dict = {}
    for deps in data.values():
        for i in deps:
            dep_dict[i] = dep_dict.get(i,0)+1

    print(f'distinct dependency file: {len(dep_dict)}')

    return rep_list,dep_list,data
    ### rep_list format :  ['https://github.com/AgriculturalModelExchangeInitiative/Crop2ML'  ... ]
    ### dep_list format: [['ipython', 'jupyter-sphinx', 'nbformat', 'nbsphinx', 'path-py', 'six', 'sphinx', 
    #                      'sphinx-hoverxref', 'sphinx-rtd-theme'], ['pypng', 'requests'],  ....] 
    ### data format: {repo1: [dep1,dep2], ...}

def d2v(dep_list):
    LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
    all_content_train = []
    j=0
    for em in dep_list:
        all_content_train.append(LabeledSentence1(em,[j]))
        j+=1
    d2v_model = Doc2Vec(all_content_train, 
                    size = 100, 
                    window = 10, 
                    min_count = 1, 
                    workers=7, 
                    dm = 1,
                    alpha=0.025, 
                    min_alpha=0.001)
    d2v_model.train(all_content_train, 
                    total_examples=d2v_model.corpus_count, 
                    epochs=10, 
                    start_alpha=0.002, 
                    end_alpha=-0.016)

    return d2v_model
    ### d2v_model can be seen as a list, each item represents a doc vector 

def kmeans(k,d2v_model,rep_list):
    kmeans_model = KMeans(n_clusters=k, init='k-means++', max_iter=500) 
    X = kmeans_model.fit(d2v_model.docvecs.doctag_syn0)
    labels=kmeans_model.labels_

    topic_dict = {}
    for index,label in enumerate(labels):
        topic_id = label
        # print(topic_id, '--->', rep_list[index])
        topic_dict[label] = topic_dict.get(label,[])
        topic_dict[label].append(rep_list[index])

    for k in sorted(topic_dict.keys()):
        print(f'topic {k} : repos num: {len(topic_dict[k])}')

    return topic_dict
    ## topic_dict is a dictionary whose key is the topic and value is a list of repos
    ## format {top1:  [repo1,repo2] ....}

def gmm(k,d2v_model):
    GMM = GaussianMixture(n_components=k).fit(d2v_model.docvecs.doctag_syn0)
    probs = GMM.predict_proba(d2v_model.docvecs.doctag_syn0)
    #probs.shape,probs
    return probs

### LDA ### 
def LDA(data,rep_list):
    # based on dep file names , build dep name dictionary
    id2word  = corpora.Dictionary(list(data.values()))   # {0: 'emd-signal',1: 'numpy', 2: 'SQLAlchemy' ...}
    # based on dep name dict and dep names, build corpus
    corpus = [id2word.doc2bow(text) for text in list(data.values())] # [[(0, 1), (1, 1)],.....]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    # pprint(lda_model.print_topics())

    print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=list(data.values()), dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: ', coherence_lda)

    # Show the top 5 words of each topic
    for topic in lda_model.print_topics(num_words=5):
        print(topic)

    # get the possible of each topic
    probs = lda_model.inference(corpus)[0]

    # inference
    topic_dict = {}
    for e, values in enumerate(lda_model.inference(corpus)[0]):
        topic_val = 0
        topic_id = 0
        for tid, val in enumerate(values):
            if val > topic_val:
                topic_val = val
                topic_id = tid        
        topic_dict[topic_id] = topic_dict.get(topic_id,[])
        topic_dict[topic_id].append(rep_list[e])

    return probs,topic_dict



if __name__ == "__main__":
    input_file = '../data/dependency_data_8k.json'
    # load data
    print('Data looking:')
    rep_list,dep_list,data = load_data(input_file)
    print('='*40)
    #doc2vector
    d2v_model = d2v(dep_list)
    # kmeans
    print('KMEANS:')
    topic_dict = kmeans(10,d2v_model,rep_list)
    print('='*40)
    # gmm
    print('GMM:')
    probs = gmm(10,d2v_model)
    print(probs)
    print('='*40)
    
    # LDA
    print('LDA:')
    probs_lds, topic_dict_lda = LDA(data,rep_list)
    print(probs_lds)
    print('='*40)


Data looking:
repos nums: 1869
repos which have no dependency files: 0
repos with dependency files: 1869
distinct dependency file: 12947


E:\Anaconda\lib\site-packages\gensim\models\doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


KMEANS:


<ipython-input-1-8caa7867c192>:79: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  X = kmeans_model.fit(d2v_model.docvecs.doctag_syn0)


topic 0 : repos num: 9
topic 1 : repos num: 1353
topic 2 : repos num: 37
topic 3 : repos num: 52
topic 4 : repos num: 40
topic 5 : repos num: 2
topic 6 : repos num: 90
topic 7 : repos num: 240
topic 8 : repos num: 14
topic 9 : repos num: 32
GMM:


<ipython-input-1-8caa7867c192>:97: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  GMM = GaussianMixture(n_components=k).fit(d2v_model.docvecs.doctag_syn0)
<ipython-input-1-8caa7867c192>:98: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  probs = GMM.predict_proba(d2v_model.docvecs.doctag_syn0)


[[9.99992923e-001 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  2.09791969e-197 0.00000000e+000]
 [0.00000000e+000 4.58184954e-211 1.00000000e+000 ... 0.00000000e+000
  3.63979587e-255 0.00000000e+000]
 [1.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  1.51789307e-168 0.00000000e+000]
 ...
 [1.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  8.48206793e-107 0.00000000e+000]
 [9.99999630e-001 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  7.23142339e-182 0.00000000e+000]
 [9.99998027e-001 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  2.25848902e-138 0.00000000e+000]]
LDA:
Perplexity:  -8.75247979015459
Coherence Score:  0.6670300120458954
(0, '0.029*"pylint" + 0.023*"xarray" + 0.012*"protobuf" + 0.010*"sphinx-autodoc-typehints" + 0.009*"mypy"')
(1, '0.006*"@babel/parser" + 0.006*"@babel/core" + 0.005*"@babel/helper-module-imports" + 0.005*"@babel/helpers" + 0.005*"@babel/helper-optimise-call-expression"')
(2, '0.026*"junit:j

In [ ]:
# In[1]:
import sys
sys.path.append(r'../xkcd2347/')
import xkcd2347
import json
from tqdm import tqdm


def getDependencies(input_file, output_file, git_hub_key, hop):
    dependency_data = {}
    no_dependency_data = []
    gitLink = []

    with open(input_file, 'r') as fp:
        gitLink = fp.readlines()

    for link in tqdm(gitLink):
        gh = xkcd2347.GitHub(key = git_hub_key)
        link = link.rstrip()
        owner, repo = link.split(".com/")[1].split("/")
        result = [dep['packageName'] for dep in gh.get_dependencies(repo_owner = owner, repo_name = repo, depth = hop)]
        if(len(result) > 0):
            dependency_data[link] = result
    with open(output_file, 'w') as fp:
        json.dump(dependency_data, fp, indent=4)

# In[ ]:


if __name__ == "__main__":
    input_file = '3k_valid_repos'
    output_file = '3k_repodata_test'
    git_hub_key = 'b3643745a923e31a34fa5f56f33b59fba810241f'
    hop = 1
    getDependencies(input_file, output_file, git_hub_key, hop)

  0%|                                                                                 | 1/1440 [00:01<26:25,  1.10s/it]

Query failed to run by returning code of 401. 
            {
              repository(owner: "pymedphys", name: "pymedphys") {
                description
                dependencyGraphManifests(first: 50) {
                  nodes {
                    blobPath
                    dependencies {
                      nodes {
                        packageName
                        repository {
                          name
                          owner {
                            login
                          }
                          primaryLanguage {
                            name
                          }
                        }
                        requirements
                        hasDependencies
                      }
                    }
                  }
                }
              }
            }
            
500 pymedphys/pymedphys


  0%|                                                                                 | 2/1440 [00:02<31:31,  1.32s/it]

Query failed to run by returning code of 401. 
            {
              repository(owner: "anuprulez", name: "galaxy_tool_recommendation") {
                description
                dependencyGraphManifests(first: 50) {
                  nodes {
                    blobPath
                    dependencies {
                      nodes {
                        packageName
                        repository {
                          name
                          owner {
                            login
                          }
                          primaryLanguage {
                            name
                          }
                        }
                        requirements
                        hasDependencies
                      }
                    }
                  }
                }
              }
            }
            
500 anuprulez/galaxy_tool_recommendation


  0%|▏                                                                                | 3/1440 [00:03<25:34,  1.07s/it]

...

Query failed to run by returning code of 401. 
            {
              repository(owner: "AdmiralenOla", name: "Scoary") {
                description
                dependencyGraphManifests(first: 50) {
                  nodes {
                    blobPath
                    dependencies {
                      nodes {
                        packageName
                        repository {
                          name
                          owner {
                            login
                          }
                          primaryLanguage {
                            name
                          }
                        }
                        requirements
                        hasDependencies
                      }
                    }
                  }
                }
              }
            }
            
500 AdmiralenOla/Scoary


  3%|██▏                                                                             | 39/1440 [00:36<27:07,  1.16s/it]

Query failed to run by returning code of 401. 
            {
              repository(owner: "NickleDave", name: "visual-search-nets") {
                description
                dependencyGraphManifests(first: 50) {
                  nodes {
                    blobPath
                    dependencies {
                      nodes {
                        packageName
                        repository {
                          name
                          owner {
                            login
                          }
                          primaryLanguage {
                            name
                          }
                        }
                        requirements
                        hasDependencies
                      }
                    }
                  }
                }
              }
            }
            
500 NickleDave/visual-search-nets


  3%|██▏                                                                             | 40/1440 [00:36<22:40,  1.03it/s]

Query failed to run by returning code of 401. 
            {
              repository(owner: "dhondta", name: "AppmemDumper") {
                description
                dependencyGraphManifests(first: 50) {
                  nodes {
                    blobPath
                    dependencies {
                      nodes {
                        packageName
                        repository {
                          name
                          owner {
                            login
                          }
                          primaryLanguage {
                            name
                          }
                        }
                        requirements
                        hasDependencies
                      }
                    }
                  }
                }
              }
            }
            
500 dhondta/AppmemDumper
